## IMDb

At Fast.ai we have introduced a new module called fastai.text which replaces the torchtext library that was used in our 2018 dl1 course. The fastai.text module also supersedes the fastai.nlp library but retains many of the key functions.

In [7]:
from fastai.text import *
from fastai.core import num_cpus, partition_by_cores
import html
from pathlib import Path
import numpy as np
import csv
import pandas as pd
from collections import Counter, defaultdict
from itertools import chain
from nltk.corpus import brown
import os

from gensim.corpora import Dictionary
from gensim.models import Word2Vec
from typing import Callable, List, Collection
from concurrent.futures.process import ProcessPoolExecutor

The Fastai.text module introduces several custom tokens.

We need to download the IMDB large movie reviews from this site: http://ai.stanford.edu/~amaas/data/sentiment/
Direct link : [Link](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz) and untar it into the PATH location. We use pathlib which makes directory traveral a breeze.

In [2]:
PATH=Path('/media/discoD/repositorios/1-billion-word-language-modeling-benchmark/')

## Standardize format

The imdb dataset has 3 classes. positive, negative and unsupervised(sentiment is unknown). 
There are 75k training reviews(12.5k pos, 12.5k neg, 50k unsup)
There are 25k validation reviews(12.5k pos, 12.5k neg & no unsup)

Refer to the README file in the imdb corpus for further information about the dataset.

In [3]:
class VocabularyTokenizer():
    "Put together rules, a tokenizer function and a language to tokenize text with multiprocessing."
    def __init__(self, tok_func:Callable=SpacyTokenizer, lang:str='pt', n_cpus:int=None):
        self.tok_func,self.lang = tok_func,lang
        self.n_cpus = n_cpus or num_cpus()//2

    def process_text(self, t:str, tok:BaseTokenizer) -> List[str]:
        "Processe one text `t` with tokenizer `tok`."
        return tok.tokenizer(t)

    def _process_all_1(self, texts:Collection[str]) -> List[List[str]]:
        "Process a list of `texts` in one process."
        tok = self.tok_func(self.lang)
        return [self.process_text(t, tok) for t in texts]

    def process_all(self, texts:Collection[str]) -> List[List[str]]:
        "Process a list of `texts`."
        if self.n_cpus <= 1: return self._process_all_1(texts)
        with ProcessPoolExecutor(self.n_cpus) as e:
            return sum(e.map(self._process_all_1, partition_by_cores(texts, self.n_cpus)), [])

In [8]:
def save_texts(paths, filename, lang):
    CLASSES = ['unsup']
    file_count = 0
    filename = filename + '_' + lang + '.csv'
    if os.path.isfile(filename):
        os.remove(filename)
    with open(filename, 'a') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONE, escapechar='\\')
        for idx,label in enumerate(CLASSES):
            for path in paths:
                for fname in (path).glob('*'):
                    file_count += 1
                    print('writing from %s' % fname)
                    [writer.writerow([line, idx]) for line in fname.open('r', encoding='utf-8').read().split('\n')]
    print('%d texts saved to %s' % (file_count, filename))

In [10]:
save_texts([PATH/'training-wiki/'], 'train', 'pt')
save_texts([PATH/'heldout-wiki/'], 'test', 'pt')
#save_texts(PATH/'training-monolingual.tokenized.shuffled/', 'train' + lang + '.csv')
#save_texts(PATH/'heldout-monolingual.tokenized.shuffled/', 'test' + lang + '.csv')
save_texts([PATH/'training-wiki/',PATH/'heldout-wiki/'], 'full', 'pt')

writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00001-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00002-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00003-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00004-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00005-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00006-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00007-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00008-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-

writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00072-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00073-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00074-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00075-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00076-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00077-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00078-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00079-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-

writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00002-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00003-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00004-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00005-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00006-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00007-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00008-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00009-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-

writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00073-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00074-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00075-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00076-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00077-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00078-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00079-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-wiki/wiki-00080-of-00100
writing from /media/discoD/repositorios/1-billion-word-language-

In [12]:
def get_tokens(filename):
    data = pd.read_csv(filename, header=None, escapechar='\\', chunksize=50000)
    for idx, df in enumerate(data):
        print(idx)
        yield VocabularyTokenizer().process_all(df[0].astype(str))
        #if idx == 2:
        #    break

In [14]:
freq_full = Counter(p for o in chain.from_iterable(get_tokens('full_pt.csv')) for p in o)
freq_full.most_common()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109


[(',', 16380912),
 ('de', 13564059),
 ('.', 9741152),
 ('a', 7056509),
 ('e', 6356054),
 ('"', 5632568),
 ('o', 5533639),
 ('do', 3910080),
 ('em', 3787678),
 ('da', 3702296),
 ('que', 3274215),
 ('um', 2376198),
 ('uma', 2256461),
 (')', 2143234),
 ('(', 2138380),
 ('com', 2067625),
 ('é', 2006873),
 ('no', 1940279),
 ('para', 1910855),
 ('na', 1738491),
 ('foi', 1548984),
 ('os', 1518939),
 ('por', 1512169),
 ('como', 1289488),
 ('O', 1128239),
 ('A', 1105913),
 ('dos', 1062039),
 ('se', 893225),
 ('as', 882122),
 ('sua', 816387),
 ('mais', 802407),
 (':', 712717),
 ('à', 707284),
 ('Em', 704036),
 ('seu', 698062),
 ('não', 631079),
 ('das', 621536),
 ('ou', 549752),
 ('pela', 530810),
 ('também', 501161),
 ('pelo', 489051),
 ('ser', 450721),
 ('d', 423274),
 ('entre', 414955),
 ('são', 401722),
 ('anos', 367910),
 ('ele', 364303),
 ('era', 346309),
 ('mas', 335829),
 ('-', 327851),
 (';', 322370),
 ('nos', 304985),
 ('foram', 304542),
 ('No', 304489),
 ('cidade', 291921),
 ('até', 2

In [15]:
palavras = [palavra for palavra, contagem in freq_full.most_common()]
palavras

[',',
 'de',
 '.',
 'a',
 'e',
 '"',
 'o',
 'do',
 'em',
 'da',
 'que',
 'um',
 'uma',
 ')',
 '(',
 'com',
 'é',
 'no',
 'para',
 'na',
 'foi',
 'os',
 'por',
 'como',
 'O',
 'A',
 'dos',
 'se',
 'as',
 'sua',
 'mais',
 ':',
 'à',
 'Em',
 'seu',
 'não',
 'das',
 'ou',
 'pela',
 'também',
 'pelo',
 'ser',
 'd',
 'entre',
 'são',
 'anos',
 'ele',
 'era',
 'mas',
 '-',
 ';',
 'nos',
 'foram',
 'No',
 'cidade',
 'até',
 'seus',
 'onde',
 'Os',
 '/',
 "'",
 'sobre',
 'tem',
 'km',
 'ano',
 'São',
 'quando',
 'parte',
 'sendo',
 'nas',
 'primeiro',
 'mesmo',
 'dois',
 'primeira',
 'suas',
 'Brasil',
 'durante',
 'depois',
 's',
 'Foi',
 'nome',
 'outros',
 'ela',
 'As',
 'ainda',
 'dia',
 'grande',
 'região',
 'este',
 'álbum',
 'família',
 'está',
 '%',
 'já',
 'muito',
 'The',
 'pode',
 'maior',
 'após',
 '--',
 '²',
 'apenas',
 'vez',
 'esta',
 'É',
 '1',
 'Ele',
 'área',
 'três',
 'estado',
 'Rio',
 'Na',
 'ter',
 'forma',
 'filme',
 'n',
 'então',
 'contra',
 'duas',
 'sido',
 'segundo'

In [16]:
len(palavras)

2386970

In [19]:
palavras.insert(0, '<UNK>')
palavras.insert(0, '<S>')
palavras.insert(0, '</S>')
palavras

['</S>',
 '<S>',
 '<UNK>',
 ',',
 'de',
 '.',
 'a',
 'e',
 '"',
 'o',
 'do',
 'em',
 'da',
 'que',
 'um',
 'uma',
 ')',
 '(',
 'com',
 'é',
 'no',
 'para',
 'na',
 'foi',
 'os',
 'por',
 'como',
 'O',
 'A',
 'dos',
 'se',
 'as',
 'sua',
 'mais',
 ':',
 'à',
 'Em',
 'seu',
 'não',
 'das',
 'ou',
 'pela',
 'também',
 'pelo',
 'ser',
 'd',
 'entre',
 'são',
 'anos',
 'ele',
 'era',
 'mas',
 '-',
 ';',
 'nos',
 'foram',
 'No',
 'cidade',
 'até',
 'seus',
 'onde',
 'Os',
 '/',
 "'",
 'sobre',
 'tem',
 'km',
 'ano',
 'São',
 'quando',
 'parte',
 'sendo',
 'nas',
 'primeiro',
 'mesmo',
 'dois',
 'primeira',
 'suas',
 'Brasil',
 'durante',
 'depois',
 's',
 'Foi',
 'nome',
 'outros',
 'ela',
 'As',
 'ainda',
 'dia',
 'grande',
 'região',
 'este',
 'álbum',
 'família',
 'está',
 '%',
 'já',
 'muito',
 'The',
 'pode',
 'maior',
 'após',
 '--',
 '²',
 'apenas',
 'vez',
 'esta',
 'É',
 '1',
 'Ele',
 'área',
 'três',
 'estado',
 'Rio',
 'Na',
 'ter',
 'forma',
 'filme',
 'n',
 'então',
 'contra',
 

In [18]:
sum(freq_full.values())

267312748

In [22]:
with open('vocabulario.txt', 'w') as vocab:
    for palavra in palavras:
        vocab.write(palavra + '\n')
vocab.close()